# Running this notebook
`bazel build Cartography:jupyter && ./bazel-bin/Cartography/jupyter`

# Contextual Building Typification in Automated Map Generalization
## Definitions
Cartographic generalization is an abstraction process that seeks to reduce the quantity of information in accordance with particular objectives. Here we care about scale, which means that the objective is to increase legibility. 

When generalizing buildings it is important to maintain:
1. Legibility
2. The visual identiy of each building
3. The pattern among a local group of buildings
4. The overal density of buildings across a region of the map.

## Legibility constraints
They ensure that the map will be readable. Legibility depends on size, contrast, brightness, etc. Legibility constraints can be divided into three classes:
1. *Perception* specifies  constraints on the sizes of objects or on their details. For instance: minimum size of a building or minimum edge size.
2. *Separation*

# Input